# **Minimum Edit Distance 3**

# **Write up: Briefly Define your Algorithm**

*Place your write up here as to how you implemented the minimum edit distance algorithm. Be as clear as you can, but don't over do it.*

**Minimum Edit Distance**<br>
I first make a matrix with the length of the target + 1 by the source + 1. I also initialize the matrix's first row and column by adding values from 0 to the length of the source + 1 for the row and the length of the target + 1 for the column. The algorithm starts at the index [1,1] wherein I check whether the character at the row index of the source matches the character at the column index of the target matches each other. If they do, then the I copy the value diagonally to the top left of it which becomes the new entry of the matrix, meaning it is a match. Otherwise, I get the values one index up + 1(insert), to the left + 1(delete), and diagonally to the top left + 2(mismatch) and choose which among the three is the minimum. Then the minimum will be the new entry in the matrix. I continue this as I traverse each index of the matrix from left to right and top to bottom. The last entry of the matrix will result to the minimum edit distance for the source and target strings.<br><br>
**Backtracing**<br>
For the back tracing portion, I had to traverse from the bottom right, to the top most index while choosing the path with the least weight. While in a loop, I choose between the value above, to the left, and diagonally to the top left whether which is the minimum. There are four conditions I have to check to know which minimum operation to use to help assign the alignments. I check for a mismatch, meaning the diagonal value is the minimum and is minus 2 of the current index, I check for a match wherein the diagonal value is the minimum and is the same as the current index, an insertion where the minium value is to the left of the current index, and lastly a delete operation which is one value above the current index. I have an operation string that keeps getting appendend with the minimum operation as I back trace throught the matrix. For the alignments, if I get a mismatch, I just copy both characters from the source and target's row and column index respectively. I do the same thing for a match. If I get an insertion, the target alignment gets a "-" character while the source alignment gets the column index of the original string. Lastly for a deletion, the source alignment gets a "-" character while the target alignment gets the character at the row index of the orginal string. 

In [ ]:

from typing import Tuple, List, Dict

def _backtrace(matrix: List[List[int]], source: str, target: str, allow_transpose: bool=False) -> Tuple[str,str,str]:
    i, j = len(source), len(target)
    s_align, t_align, ops = [], [], []
    while i>0 or j>0:
        cur = matrix[i][j]
        moved = False

        if i>0 and j>0 and matrix[i-1][j-1] + (0 if source[i-1]==target[j-1] else 1) == cur:
            s_align.append(source[i-1])
            t_align.append(target[j-1])
            ops.append(" " if source[i-1]==target[j-1] else "M")
            i -= 1; j -= 1; moved=True
    
        elif allow_transpose and i>1 and j>1 and source[i-1]==target[j-2] and source[i-2]==target[j-1] and matrix[i-2][j-2] + 1 == cur:
            s_align.append(source[i-2:i])
            t_align.append(target[j-2:j])
            ops.append("T")
            i -= 2; j -= 2; moved=True
 
        elif j>0 and matrix[i][j-1] + 1 == cur:
            s_align.append("-")
            t_align.append(target[j-1])
            ops.append("I")
            j -= 1; moved=True
    
        elif i>0 and matrix[i-1][j] + 1 == cur:
            s_align.append(source[i-1])
            t_align.append("-")
            ops.append("D")
            i -= 1; moved=True
        if not moved: 
            if i>0 and j>0:
                s_align.append(source[i-1]); t_align.append(target[j-1]); ops.append("?")
                i-=1; j-=1
            elif i>0:
                s_align.append(source[i-1]); t_align.append("-"); ops.append("D"); i-=1
            else:
                s_align.append("-"); t_align.append(target[j-1]); ops.append("I"); j-=1
    return "".join(reversed(ops)), "".join(reversed(s_align)), "".join(reversed(t_align))

def getMinEditDistance(source: str, target: str, allow_transpose: bool=False) -> Tuple[int, str, str, str]:
    """
    Computes Levenshtein Minimum Edit Distance (unit costs).
    If allow_transpose=True, uses the optimal string alignment (restricted Damerau) distance
    where adjacent transpositions cost 1.
    Returns: (distance, ops, s_alignment, t_alignment)
    Legend: I=Insert, D=Delete, M=Mismatch(Substitution), ' '=Match, T=Transpose (if enabled)
    """
    m, n = len(source), len(target)
    
    matrix = [[0]*(n+1) for _ in range(m+1)]
    for i in range(1, m+1): matrix[i][0] = i
    for j in range(1, n+1): matrix[0][j] = j

    for i in range(1, m+1):
        for j in range(1, n+1):
            cost = 0 if source[i-1]==target[j-1] else 1
            matrix[i][j] = min(
                matrix[i-1][j] + 1,     
                matrix[i][j-1] + 1,     
                matrix[i-1][j-1] + cost  
            )
            if allow_transpose and i>1 and j>1 and source[i-1]==target[j-2] and source[i-2]==target[j-1]:
                matrix[i][j] = min(matrix[i][j], matrix[i-2][j-2] + 1)

    ops, s_align, t_align = _backtrace(matrix, source, target, allow_transpose)
    dist = matrix[m][n]

    print(f"Distance: {dist}")
    print(ops)
    print(s_align)
    print(t_align)
    return dist, ops, s_align, t_align


[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [10, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [11, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [12, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
 
 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 
 [2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 
 [3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 
 [4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 
 [5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 
 [6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 
 [7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 
 [8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 
 [9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 
 [10, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 
 [11, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 
 [12, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

#**Test Cases**

**Legend**<br>
I - Insert <br>
D - Delete <br>
M - Mismatch <br>
" " - Match

In [4]:
getMinEditDistance("naruto's son", "boruto's dad")

Source: naruto's son
Target: boruto's dad
-----
Distance:  10
Aligment:
naruto's son
MM       MMM
boruto's dad


In [5]:
getMinEditDistance("kumakain", "kumain")

Source: kumakain
Target: kumain
-----
Distance:  2
Aligment:
kumakain
    DD  
kuma--in


In [6]:
getMinEditDistance("levinstien", "levenshtein")

Source: levinstien
Target: levenshtein
-----
Distance:  5
Aligment:
levins-tie-n
   M  I D I 
levensht-ein


In [7]:
getMinEditDistance("leviathan", "levenshtein")

Source: leviathan
Target: levenshtein
-----
Distance:  10
Aligment:
lev--iathan
   IIMM MM 
levenshtein


In [8]:
getMinEditDistance("ATGCATCCCATGAC", "TCTATATCCGT")

Source: ATGCATCCCATGAC
Target: TCTATATCCGT
-----
Distance:  11
Aligment:
ATGC-ATCCCAT--GAC
D D I  DDD  II DM
-T-CTAT---ATCCG-T


In [9]:
getMinEditDistance("AGGCTATCACCTGACCTCCAGGCCGATGCCCACCTGG", "TAGCTATCACGACCGCGGTCGATTTGCCCGACGGTCC")

Source: AGGCTATCACCTGACCTCCAGGCCGATGCCCACCTGG
Target: TAGCTATCACGACCGCGGTCGATTTGCCCGACGGTCC
-----
Distance:  18
Aligment:
-AGGCTATCACCTGACCTCCAGG-CCGAT--GCCC-ACCTGG---
I  D       DD    M DD  I D   II    I  DD  III
TAG-CTATCAC--GACCGC--GGTC-GATTTGCCCGAC--GGTCC


### Your tasks-1:

Run the function getMinEditDistance(source, target) on the following test cases:

* "kitten" → "sitting"

* "flaw" → "lawn"

* "intention" → "execution"

Write down:

* the edit distance,

* the sequence of operations (Insertion I, Deletion D, Mismatch M),

* and the final alignment printed.

### Your tasks-2:
Extend the algorithm:
Modify the function so that:

Insertion = 1

Deletion = 1

Substitution = 1

(Bonus) Transposition of two adjacent characters = 1

Test your new algorithm on:

"abcd" → "acbd"

In [ ]:

getMinEditDistance("abcd", "acbd", allow_transpose=True)
